In [3]:
# --- IMPORTS FOR EDA ---
import pandas as pd
import numpy as np
import umap.umap_ as umap
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
import torch # Import PyTorch

# --- DATA LOADING ---
df_normals = pd.read_csv(r'C:\Users\noaga\OneDrive\Desktop\2025-6\AI Enhanced cyber\Project\Splited_data\01_train_normal.csv')
df_val_normal = pd.read_csv(r'C:\Users\noaga\OneDrive\Desktop\2025-6\AI Enhanced cyber\Project\Splited_data\02_validation_normal.csv')
df_test_all = pd.read_csv(r'C:\Users\noaga\OneDrive\Desktop\2025-6\AI Enhanced cyber\Project\Splited_data\03_test_all.csv')

print(f"Normals dataset: set size: {len(df_normals)}")
print(f"Validation (Normal) set size: {len(df_val_normal)}")
print(f"Test (Mixed) set size: {len(df_test_all)}")

Normals dataset: set size: 313176
Validation (Normal) set size: 67109
Test (Mixed) set size: 96877


In [44]:
#--- Normals dataset ---
anomalies_count = (df_normals['is_phishing'] == 1).sum()
print("Anomalies:", anomalies_count)
normal_count = (df_normals['is_phishing'] == 0).sum()
print("Normal behaivour:", normal_count)

Anomalies: 0
Normal behaivour: 313176


In [45]:
#--- validation dataset---
anomalies_count = (df_val_normal['is_phishing'] == 1).sum()
print("Anomalies:", anomalies_count)
normal_count = (df_val_normal['is_phishing'] == 0).sum()
print("Normal behaivour:", normal_count)

Anomalies: 0
Normal behaivour: 67109


In [46]:
#--- Test dataset ---
anomalies_count = (df_test_all['is_phishing'] == 1).sum()
print("Anomalies:", anomalies_count)
normal_count = (df_test_all['is_phishing'] == 0).sum()
print("Normal behaivour:", normal_count)


Anomalies: 29767
Normal behaivour: 67110
